In [29]:
import pandas as pd
import yfinance as yf

symbols = ['GOOG', 'AAPL', 'MSFT']

tickers = [sym.replace('.','-') for sym in symbols]
symbols_str = ' '.join(tickers)
    
data = yf.download(symbols_str, period='5d', interval='1d', group_by='ticker', progress=False, auto_adjust=False)

print(data)
        
prices = {}

try:
    if len(symbols) == 1:
        # handle yfinance MultiIndex columns (ticker, field) and plain columns
        if isinstance(data.columns, pd.MultiIndex):
            # use ticker name (tickers list) as first level
            close_series = data[(tickers[0], 'Close')]
        if 'Close' in data.columns:
            close_series = data['Close']
        last_close = close_series.dropna().iloc[-1]
        prices[symbols[0]] = float(last_close)

    else:
        for symbol in symbols:
            if isinstance(data.columns, pd.MultiIndex) and symbol in data.columns.get_level_values(0):
                price = data[(symbol, 'Close')].dropna().iloc[-1]
                prices[symbol] = float(price)
            else:
                # data['Close'] may be a DataFrame with symbols as columns
                if 'Close' in data.columns and symbol in data['Close'].columns:
                    price = data['Close'][symbol].dropna().iloc[-1]
                    prices[symbol] = float(price)
    return prices
except Exception as e:
    print("Error getting close for", symbols[0], e)



Ticker            GOOG                                                  \
Price             Open        High         Low       Close   Adj Close   
Date                                                                     
2026-01-23  332.654999  334.029999  327.970001  328.429993  328.429993   
2026-01-26  328.200012  336.350006  327.489990  333.589996  333.589996   
2026-01-27  335.630005  338.214996  333.750000  335.000000  335.000000   
2026-01-28  336.609985  337.630005  332.130005  336.279999  336.279999   
2026-01-29  340.545013  342.290009  326.720001  338.660004  338.660004   

Ticker                      MSFT                                      \
Price         Volume        Open        High         Low       Close   
Date                                                                   
2026-01-23  19110600  451.869995  471.100006  450.529999  465.950012   
2026-01-26  18502500  465.309998  474.250000  462.000000  470.279999   
2026-01-27  16080900  473.700012  482.869995  4

SyntaxError: 'return' outside function (2295858987.py, line 36)

OperationalError: unable to open database file

In [2]:
import sqlite3 as sq

conn = sq.connect('src/portfolios/database/portfolios.db')
cur = conn.cursor()

cur.execute('''
        ALTER TABLE transactions ADD COLUMN sector TEXT NOT NULL DEFAULT 'N/A';
''')

conn.commit()
conn.close()

OperationalError: unable to open database file

In [ ]:

import yfinance as yf
import pandas as pd

def get_batch_prices(symbols, price_change=False, compare_to='day'):
    """
    batch processing to get stock price data for multiple symbols.
    Use price_change=True to get percent change vs compare_to price.
    
    :param symbols: list of symbols
    :param price_change: bool, whether to compute price change info
    :param compare_to: 'week' or 'day' for price comparison

    :return: dict of symbol to last close price, or dict with price change info

    """
    tickers = [sym for sym in symbols]
    symbols_str = ' '.join(tickers)
        
    data = yf.download(symbols_str, period='5d', interval='1d', group_by='ticker', progress=False, auto_adjust=False)
            
    prices = {}

    try:
        if len(symbols) == 1:
            if isinstance(data.columns, pd.MultiIndex):
                close_series = data[symbols[0]]['Close']

            last_close = close_series.iloc[-1]

            if price_change:
                if compare_to == 'day':
                    compare_price = close_series.iloc[-2]
                elif compare_to == 'week':
                    compare_price = close_series.iloc[-5]
                else:
                    raise ValueError("Invalid compare_to value. Use 'day' or 'week'.")

                percentage_change, change = 'working', 'working'

                prices[symbols[0]] = {
                    'last_close': float(last_close),
                    'compare_price': float(compare_price),
                    'change': change,
                    'percentage_change': percentage_change
                }
            else:
                prices[symbols[0]] = float(last_close)

        else:
            for symbol in symbols:
                if isinstance(data.columns, pd.MultiIndex):
                    close_series = data[symbol]['Close']

                last_close = close_series.iloc[-1]

                if price_change:
                    if compare_to == 'day':
                        compare_price = close_series.iloc[-2]
                    elif compare_to == 'week':
                        compare_price = close_series.iloc[-5]
                    else:
                        raise ValueError("Invalid compare_to value. Use 'day' or 'week'.")
                    
                    percentage_change, change = 'working', 'working'

                    prices[symbol] = {
                        'last_close': float(last_close),
                        'compare_price': float(compare_price),
                        'change': change,
                        'percentage_change': percentage_change
                    }
                else:
                    prices[symbol] = float(last_close)

        return prices
    
    except Exception as e:
        print(f"Error getting close for {symbol}: {e}")
        return {}

symb = ['AAPL', 'GOOG']
print(get_batch_prices(symb, price_change=True, compare_to='week'))

{'AAPL': {'last_close': 275.9100036621094, 'compare_price': 259.4800109863281, 'change': 'working', 'percentage_change': 'working'}, 'GOOG': {'last_close': 331.3299865722656, 'compare_price': 338.5299987792969, 'change': 'working', 'percentage_change': 'working'}}
